In [1]:
#imports
from tbselenium.tbdriver import TorBrowserDriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import os
import time
from bs4 import BeautifulSoup as bs

In [2]:
#list of working URLs to the marketplace
URLS= ['http://p5eg3xsp2u3typmi3vk6crlohprr6dkhk3jevr4r7i44myv6nugea3qd.onion/',
      'http://4xt3ytgjiapwbzd2pp2ofrhr4rxygxe4vlrwjjsr4il3zues6xzirtad.onion/']

#link to search all vendors
VENDORS = ['http://4xt3ytgjiapwbzd2pp2ofrhr4rxygxe4vlrwjjsr4il3zues6xzirtad.onion/vendors?vendorsearch=&vendor_search_min_lvl=&is_fe_allowed=&vendors_orderby=vendorsincedesc']
VENDORS_URL = 'vendors?vendors_orderby=vendorsincedesc'


def visit_and_login(driver):
    '''This function opens the marketplace in the url'''
    
    # Log in and solve the captcha
    driver.get(URLS[1])
    time.sleep(30) #30 seconds to solve the captcha

    return driver

urls=[]

def get_pages(driver, urls):
    print("Starting the crawling")
    for i in range(2, 27):
        site_page = URLS[1] + VENDORS_URL + '&page=' + str(i)
        driver.get(site_page)
        time.sleep(2)
        
        soup = bs(driver.page_source, 'html.parser')
        
        hrefs = soup.find_all(href=True) #find all linked urls
        
        for line in hrefs:
            urls.append(line['href']) #append all urls to list



        
                
#         filename = 'darkfox/'+ 'page'+str(i) + '.html'
#         with open("/home/levi/" + filename, "w") as f:
#             f.write(driver.page_source)
#             print('dowloaded',i, 'page')
    print('Finished Crawling')
    return urls

In [ ]:
with TorBrowserDriver("/home/levi/tor-browser_en-US/") as driver:
    driver = visit_and_login(driver)
    test = get_pages(driver, urls)

Starting the crawling


In [3]:
#### SCRAPE THE VENDOR URLS FROM THE DOWLOADED PAGES ####
#list that will contain urls
urls = []
vendor_urls=[]

#set directory containing html pages
DIR = '/home/levi/darkfox'

#for each html page
for page in os.listdir(DIR):
    
    html_file = os.path.join(DIR, page)#link to html
    
    #open file with beautiful soup
    with open(html_file) as f:
        soup = bs(f, 'html.parser')
        
        hrefs = soup.find_all(href=True) #find all linked urls
        
        for line in hrefs:
            urls.append(line['href']) #append all urls to list

for i in urls:
    if 'user' in i:
        vendor_urls.append(i)
vendor_urls #final list of vendor urls

['http://4xt3ytgjiapwbzd2pp2ofrhr4rxygxe4vlrwjjsr4il3zues6xzirtad.onion/user/bulkversion',
 'http://4xt3ytgjiapwbzd2pp2ofrhr4rxygxe4vlrwjjsr4il3zues6xzirtad.onion/user/RockStarPILSNL',
 'http://4xt3ytgjiapwbzd2pp2ofrhr4rxygxe4vlrwjjsr4il3zues6xzirtad.onion/user/NewYorker',
 'http://4xt3ytgjiapwbzd2pp2ofrhr4rxygxe4vlrwjjsr4il3zues6xzirtad.onion/user/blackmarket',
 'http://4xt3ytgjiapwbzd2pp2ofrhr4rxygxe4vlrwjjsr4il3zues6xzirtad.onion/user/polarsprings',
 'http://4xt3ytgjiapwbzd2pp2ofrhr4rxygxe4vlrwjjsr4il3zues6xzirtad.onion/user/QualiMoon',
 'http://4xt3ytgjiapwbzd2pp2ofrhr4rxygxe4vlrwjjsr4il3zues6xzirtad.onion/user/Panacea05',
 'http://4xt3ytgjiapwbzd2pp2ofrhr4rxygxe4vlrwjjsr4il3zues6xzirtad.onion/user/TheCheekyGirls',
 'http://4xt3ytgjiapwbzd2pp2ofrhr4rxygxe4vlrwjjsr4il3zues6xzirtad.onion/user/HoneyPotFarms',
 'http://4xt3ytgjiapwbzd2pp2ofrhr4rxygxe4vlrwjjsr4il3zues6xzirtad.onion/user/quasi',
 'http://4xt3ytgjiapwbzd2pp2ofrhr4rxygxe4vlrwjjsr4il3zues6xzirtad.onion/user/bud4less',
 'htt

In [11]:
#when crawling in multiple stages, this function updates the vendor_urls list

def links_to_crawl(DIR, vendor_urls):
    '''check in directory which pages already have been crawled and make new list with urls to crawl'''
    
    crawled = os.listdir(DIR)
    crawled = [i[:-5] for i in crawled]
    
    print('already crawled pages:' , len(crawled))

    all_vendors = [i[75:] for i in vendor_urls] #names of all vendors


    #all vendors remaining to crawl
    to_crawl = [vendor for vendor in all_vendors if vendor not in crawled]
    site = 'http://4xt3ytgjiapwbzd2pp2ofrhr4rxygxe4vlrwjjsr4il3zues6xzirtad.onion/user/'

    vendors_to_crawl = [site + vendor for vendor in to_crawl] #list of pages to crawl
    print('pages left to crawl' , len(vendors_to_crawl))
    
    return vendors_to_crawl

In [19]:
### DOWNLOAD THE ACTUAL VENDOR PAGES###

def get_vendor_pages(driver, vendor_urls):
    print("Starting the crawling")
    
  
    for idx,i in enumerate(vendor_urls): #for all vendors
        
        
        site_page = i+'/products' #visit listings of vendor
        driver.get(site_page)
        time.sleep(3)

        filename = 'darkfox/'+ str(i[75:]) + '.html' #save file als username
        with open("/home/levi/" + filename, "w") as f:
            f.write(driver.page_source)
            
            #show progress
            print('dowloaded' ,idx+1, 'out of', len(vendor_urls),'pages')
        
#go to website, login and start crawling              
with TorBrowserDriver("/home/levi/tor-browser_en-US/") as driver:
    driver = visit_and_login(driver)
    get_vendor_pages(driver, vendor_urls)

Starting the crawling
dowloaded 1 out of 541 pages
dowloaded 1 out of 541 pages
dowloaded 1 out of 541 pages
dowloaded 1 out of 541 pages
dowloaded 1 out of 541 pages
dowloaded 1 out of 541 pages
dowloaded 1 out of 541 pages
dowloaded 1 out of 541 pages
dowloaded 1 out of 541 pages
dowloaded 1 out of 541 pages
dowloaded 1 out of 541 pages
dowloaded 1 out of 541 pages
dowloaded 1 out of 541 pages
dowloaded 1 out of 541 pages
dowloaded 1 out of 541 pages
dowloaded 1 out of 541 pages
dowloaded 1 out of 541 pages
dowloaded 1 out of 541 pages
dowloaded 1 out of 541 pages
dowloaded 1 out of 541 pages
dowloaded 1 out of 541 pages
dowloaded 1 out of 541 pages
dowloaded 1 out of 541 pages
dowloaded 1 out of 541 pages
dowloaded 1 out of 541 pages
dowloaded 1 out of 541 pages
dowloaded 1 out of 541 pages
dowloaded 1 out of 541 pages
dowloaded 1 out of 541 pages
dowloaded 1 out of 541 pages
dowloaded 1 out of 541 pages
dowloaded 1 out of 541 pages
dowloaded 1 out of 541 pages
dowloaded 1 out of 54

dowloaded 1 out of 541 pages
dowloaded 1 out of 541 pages
dowloaded 1 out of 541 pages
dowloaded 1 out of 541 pages
dowloaded 1 out of 541 pages
dowloaded 1 out of 541 pages
dowloaded 1 out of 541 pages
dowloaded 1 out of 541 pages
dowloaded 1 out of 541 pages
dowloaded 1 out of 541 pages
dowloaded 1 out of 541 pages
dowloaded 1 out of 541 pages
dowloaded 1 out of 541 pages
dowloaded 1 out of 541 pages
dowloaded 1 out of 541 pages
dowloaded 1 out of 541 pages
dowloaded 1 out of 541 pages
dowloaded 1 out of 541 pages
dowloaded 1 out of 541 pages
dowloaded 1 out of 541 pages
dowloaded 1 out of 541 pages
dowloaded 1 out of 541 pages
dowloaded 1 out of 541 pages
dowloaded 1 out of 541 pages
dowloaded 1 out of 541 pages
dowloaded 1 out of 541 pages
dowloaded 1 out of 541 pages
dowloaded 1 out of 541 pages
dowloaded 1 out of 541 pages
dowloaded 1 out of 541 pages
dowloaded 1 out of 541 pages
dowloaded 1 out of 541 pages
dowloaded 1 out of 541 pages
dowloaded 1 out of 541 pages
dowloaded 1 ou

In [34]:
vendor_urls = links_to_crawl(DIR, vendor_urls)

already crawled pages: 319
pages left to crawl 0
